In [84]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt

from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from keras import layers
from keras.layers import Input, Dense
from keras.models import Model, Sequential 
from keras import regularizers

In [85]:
PATH_TO_DATA = r'C:\Users\naman\OneDrive\Desktop\Deep Learning\creditcard.csv'
card_df  = pd.read_csv(PATH_TO_DATA,header=0)

In [86]:
card_df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [87]:
card_df['Time'] = card_df['Time'].apply(lambda t: (t/3600) % 24 )

In [88]:
normal_trans = card_df[card_df['Class'] == 0].sample(4000)
fraud_trans = card_df[card_df['Class'] == 1]

In [89]:
reduced_set = pd.concat([normal_trans, fraud_trans], ignore_index=True)

In [90]:
print(f"Cleansed dataset shape : {reduced_set.shape}")

Cleansed dataset shape : (4492, 31)


In [91]:
y = reduced_set['Class']
X = reduced_set.drop('Class', axis=1)

In [92]:
print(f"Shape of Features : {X.shape} and Target: {y.shape}")

Shape of Features : (4492, 30) and Target: (4492,)


In [93]:
scaler = RobustScaler().fit_transform(X)

# Scaled data
X_scaled_normal = scaler[y == 0]
X_scaled_fraud = scaler[y == 1]

In [94]:
print(f"Shape of the input data : {X.shape[1]}")

Shape of the input data : 30


In [95]:
input_layer = Input(shape = (X.shape[1], ))

# Construct encoder network
encoded = Dense(100, activation= 'tanh', activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoded = Dense(50, activation='tanh', activity_regularizer=regularizers.l1(10e-5))(encoded)
encoded = Dense(25, activation='tanh', activity_regularizer=regularizers.l1(10e-5))(encoded)
encoded = Dense(12, activation = 'tanh', activity_regularizer=regularizers.l1(10e-5))(encoded)
encoded = Dense(6, activation='relu')(encoded)

# Decoder network
decoded = Dense(12, activation='tanh')(encoded)
decoded = Dense(25, activation='tanh')(decoded)
decoded = Dense(50, activation='tanh')(decoded)
decoded = Dense(100, activation='tanh')(decoded)

output_layer = Dense(X.shape[1], activation='relu')(decoded)

# Building a model
auto_encoder = Model(input_layer, output_layer)

In [96]:
auto_encoder.compile(optimizer='adadelta', loss='mse')

# Training the auto encoder model
auto_encoder.fit(X_scaled_normal, X_scaled_normal, batch_size=32, epochs=20, shuffle=True, validation_split=0.20)

Epoch 1/20
100/100 [==============================] - 2s 11ms/step - loss: 1.6000 - val_loss: 1.3666
Epoch 2/20
100/100 [==============================] - 1s 9ms/step - loss: 1.5996 - val_loss: 1.3663
Epoch 3/20
100/100 [==============================] - 1s 8ms/step - loss: 1.5993 - val_loss: 1.3659
Epoch 4/20
100/100 [==============================] - 1s 9ms/step - loss: 1.5989 - val_loss: 1.3655
Epoch 5/20
100/100 [==============================] - 1s 9ms/step - loss: 1.5985 - val_loss: 1.3652
Epoch 6/20
100/100 [==============================] - 1s 9ms/step - loss: 1.5982 - val_loss: 1.3649
Epoch 7/20
100/100 [==============================] - 1s 9ms/step - loss: 1.5979 - val_loss: 1.3645
Epoch 8/20
100/100 [==============================] - 1s 11ms/step - loss: 1.5975 - val_loss: 1.3642
Epoch 9/20
100/100 [==============================] - 1s 8ms/step - loss: 1.5972 - val_loss: 1.3639
Epoch 10/20
100/100 [==============================] - 1s 8ms/step - loss: 1.5969 - val_loss: 1.36

In [97]:
latent_model = Sequential()
latent_model.add(auto_encoder.layers[0])
latent_model.add(auto_encoder.layers[1])
latent_model.add(auto_encoder.layers[2])
latent_model.add(auto_encoder.layers[3])
latent_model.add(auto_encoder.layers[4])

In [98]:
normal_tran_points = latent_model.predict(X_scaled_normal)
fraud_tran_points = latent_model.predict(X_scaled_fraud)
# Making as a one collection
encoded_X = np.append(normal_tran_points, fraud_tran_points, axis=0)
y_normal = np.zeros(normal_tran_points.shape[0])
y_fraud = np.ones(fraud_tran_points.shape[0])
encoded_y = np.append(y_normal, y_fraud, axis=0)

16/16 [==============================] - 0s 4ms/step


In [99]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_enc_train, X_enc_test, y_enc_train, y_enc_test = train_test_split(encoded_X, encoded_y, test_size=0.3)

In [100]:
print(f"Encoded train data X: {X_enc_train.shape}, Y: {y_enc_train.shape}, X_test :{X_enc_test.shape}, Y_test: {y_enc_test.shape}")
print(f"Actual train & test data X: {X_train.shape}, Y: {X_train.shape}, X_test :{X_test.shape}, Y_test: {y_test.shape}")

Encoded train data X: (3144, 12), Y: (3144,), X_test :(1348, 12), Y_test: (1348,)
Actual train & test data X: (3144, 30), Y: (3144, 30), X_test :(1348, 30), Y_test: (1348,)


In [101]:
svc_clf = SVC()

svc_clf.fit(X_train, y_train)

svc_predictions = svc_clf.predict(X_test)

In [102]:
print("Classification report \n {0}".format(classification_report(y_test, svc_predictions)))

Classification report 
               precision    recall  f1-score   support

           0       0.93      1.00      0.96      1190
           1       1.00      0.45      0.62       158

    accuracy                           0.94      1348
   macro avg       0.97      0.72      0.79      1348
weighted avg       0.94      0.94      0.92      1348



In [103]:
print("Accuracy of SVC \n {:.2f}".format(accuracy_score(y_test, svc_predictions)))

Accuracy of SVC 
 0.94


In [104]:
lr_clf = LogisticRegression()

lr_clf.fit(X_enc_train, y_enc_train)

# Predict the Test data
predictions = lr_clf.predict(X_enc_test)

In [105]:
print("Classification report \n {0}".format(classification_report(y_enc_test, predictions)))

Classification report 
               precision    recall  f1-score   support

         0.0       0.96      1.00      0.98      1216
         1.0       0.96      0.65      0.77       132

    accuracy                           0.96      1348
   macro avg       0.96      0.82      0.88      1348
weighted avg       0.96      0.96      0.96      1348



In [106]:
print("Accuracy score is : {:.2f}".format(accuracy_score(y_enc_test, predictions)))

Accuracy score is : 0.96
